In [7]:
MAX_PRODUCTS = 3
MAX_PRODUCT_NAME_LENGTH = 10
MAX_DISCOUNT_RULES = 4
GIFT_WRAP_FEE = 1
SHIPPING_FEE_PER_PACKAGE = 5
ITEMS_PER_PACKAGE = 10

class Product:
    def __init__(self):
        self.name = ""
        self.price = 0
        self.giftWrap = False

class DiscountRule:
    def __init__(self):
        self.name = ""
        self.threshold = 0
        self.discountAmount = 0
        self.discountPercentage = 0
        self.quantityThreshold = 0
        self.singleProductThreshold = 0

def initializeCatalog(products):
    products[0].name = "Product A"
    products[0].price = 20

    products[1].name = "Product B"
    products[1].price = 40

    products[2].name = "Product C"
    products[2].price = 50

def initializeDiscountRules(discountRules):
    discountRules[0].name = "flat_10_discount"
    discountRules[0].threshold = 200
    discountRules[0].discountAmount = 10

    discountRules[1].name = "bulk_5_discount"
    discountRules[1].threshold = 10
    discountRules[1].discountPercentage = 5

    discountRules[2].name = "bulk_10_discount"
    discountRules[2].threshold = 20
    discountRules[2].discountPercentage = 10

    discountRules[3].name = "tiered_50_discount"
    discountRules[3].quantityThreshold = 30
    discountRules[3].singleProductThreshold = 15
    discountRules[3].discountPercentage = 50

def findProductIndex(products, productName):
    for i in range(MAX_PRODUCTS):
        if products[i].name == productName:
            return i
    return -1

def calculateSubtotal(quantities, products):
    subtotal = 0
    for i in range(MAX_PRODUCTS):
        subtotal += quantities[i] * products[i].price
    return subtotal

def calculateDiscount(quantities, discountRules, subtotal):
    discountName = None
    maxDiscount = 0

    for i in range(MAX_DISCOUNT_RULES):
        if discountRules[i].name == "flat_10_discount" and subtotal > discountRules[i].threshold:
            if discountRules[i].discountAmount > maxDiscount:
                maxDiscount = discountRules[i].discountAmount
                discountName = discountRules[i].name
        elif discountRules[i].name == "bulk_5_discount" or discountRules[i].name == "bulk_10_discount":
            for j in range(MAX_PRODUCTS):
                if quantities[j] > discountRules[i].threshold:
                    if discountRules[i].discountPercentage > maxDiscount:
                        maxDiscount = discountRules[i].discountPercentage
                        discountName = discountRules[i].name
                    break
        elif discountRules[i].name == "tiered_50_discount":
            tieredDiscountQuantity = 0
            for j in range(MAX_PRODUCTS):
                if quantities[j] > tieredDiscountQuantity:
                    tieredDiscountQuantity = quantities[j]
            if subtotal > discountRules[i].threshold and tieredDiscountQuantity > discountRules[i].quantityThreshold:
                for j in range(MAX_PRODUCTS):
                    if quantities[j] > discountRules[i].singleProductThreshold:
                        if discountRules[i].discountPercentage > maxDiscount:
                            maxDiscount = discountRules[i].discountPercentage
                            discountName = discountRules[i].name
                        break

    return discountName

def calculateDiscountAmount(quantities, discountRules, discountName, subtotal, products):
    if discountName is None:
        return 0

    for i in range(MAX_DISCOUNT_RULES):
        if discountRules[i].name == discountName:
            if discountName == "flat_10_discount":
                return discountRules[i].discountAmount
            elif discountName == "bulk_5_discount" or discountName == "bulk_10_discount":
                return (subtotal * discountRules[i].discountPercentage) // 100
            elif discountName == "tiered_50_discount":
                tieredDiscountQuantity = 0
                for j in range(MAX_PRODUCTS):
                    if quantities[j] > tieredDiscountQuantity:
                        tieredDiscountQuantity = quantities[j]
                discountedQuantity = 0 if tieredDiscountQuantity < discountRules[i].singleProductThreshold else tieredDiscountQuantity - discountRules[i].singleProductThreshold
                return discountedQuantity * products[j].price * (discountRules[i].discountPercentage / 100)

    return 0


def calculateShippingFee(totalQuantity):
    return ((totalQuantity - 1) // ITEMS_PER_PACKAGE) * SHIPPING_FEE_PER_PACKAGE

def calculateTotal(subtotal, discountAmount, shippingFee, totalQuantity):
    return subtotal - discountAmount + shippingFee + (GIFT_WRAP_FEE * totalQuantity)

products = [Product() for _ in range(MAX_PRODUCTS)]
discountRules = [DiscountRule() for _ in range(MAX_DISCOUNT_RULES)]

initializeCatalog(products)
initializeDiscountRules(discountRules)

quantities = [0] * MAX_PRODUCTS
totalQuantity = 0

for i in range(MAX_PRODUCTS):
    quantities[i] = int(input("Enter the quantity of " + products[i].name + ": "))
    totalQuantity += quantities[i]

    giftWrapOption = int(input("Is " + products[i].name + " wrapped as a gift? (0 for No, 1 for Yes): "))
    products[i].giftWrap = (giftWrapOption == 1)

discountName = calculateDiscount(quantities, discountRules, calculateSubtotal(quantities, products))
discountAmount = calculateDiscountAmount(quantities, discountRules, discountName, calculateSubtotal(quantities, products), products)
subtotal = calculateSubtotal(quantities, products)
shippingFee = calculateShippingFee(totalQuantity)
totalAmount = calculateTotal(subtotal, discountAmount, shippingFee, totalQuantity)

print("\n--- Invoice ---")
for i in range(MAX_PRODUCTS):
    productTotal = quantities[i] * products[i].price
    print(products[i].name + ": Quantity: " + str(quantities[i]) + ", Total: $" + str(productTotal) + ", Gift Wrap: " + ("Yes" if products[i].giftWrap else "No"))

print("\nSubtotal: $" + str(subtotal))

if discountName is not None:
    print("Discount applied: " + discountName + ", Amount: $" + str(discountAmount))

print("Shipping fee: $" + str(shippingFee))
print("Gift wrap fee: $" + str(GIFT_WRAP_FEE * totalQuantity))
print("Total: $" + str(totalAmount))


Enter the quantity of Product A: 45
Is Product A wrapped as a gift? (0 for No, 1 for Yes): 1
Enter the quantity of Product B: 67
Is Product B wrapped as a gift? (0 for No, 1 for Yes): 0
Enter the quantity of Product C: 34
Is Product C wrapped as a gift? (0 for No, 1 for Yes): 1

--- Invoice ---
Product A: Quantity: 45, Total: $900, Gift Wrap: Yes
Product B: Quantity: 67, Total: $2680, Gift Wrap: No
Product C: Quantity: 34, Total: $1700, Gift Wrap: Yes

Subtotal: $5280
Discount applied: tiered_50_discount, Amount: $1300.0
Shipping fee: $70
Gift wrap fee: $146
Total: $4196.0
